In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load data
users = pd.read_csv('users.csv')
meals = pd.read_csv('meal_data(new).csv')

# Preprocessing
meals.fillna({'Common_Allergens': 'None', 'Diet_Preference': 'None'}, inplace=True)
users.fillna({'Allergens': 'None', 'Diet_Preference': 'None', 'Health_Restriction': 'None'}, inplace=True)  # FIX HERE

# Create text features for meals
meals['tags'] = meals['Diet_Preference'] + ' ' + meals['food_category'] + ' ' + meals['meal_types']

# Initialize TF-IDF
tfidf = TfidfVectorizer(stop_words='english', lowercase=True)
tfidf_matrix = tfidf.fit_transform(meals['tags'])

def recommend_meals(user_id, n=5):
    try:
        user = users[users['UserID'] == user_id].iloc[0]
    except IndexError:
        return pd.DataFrame()

    filtered = meals.copy()
    
    # 1. Allergen filter
    user_allergens = [a.strip().lower() for a in user['Allergens'].split(',') if a.strip().lower() != 'none']
    if user_allergens:
        filtered = filtered[~filtered['Common_Allergens'].str.lower().str.contains(
            '|'.join(user_allergens), na=False, regex=False
        )]
    
    # 2. Health restriction filter (FIXED)
    health_map = {'Heart': 'Heart_Healthy', 'Diabetic': 'Diabetic_Friendly'}
    if user['Health_Restriction'] != 'None':
        health_col = health_map.get(user['Health_Restriction'], user['Health_Restriction'])
        filtered = filtered[filtered[health_col] == 1]
    
    # 3. Diet preference filter
    user_diets = [d.strip().lower() for d in str(user['Diet_Preference']).split(',') if d.strip().lower() != 'none']
    if user_diets:
        filtered = filtered[filtered['Diet_Preference'].str.lower().str.contains(
            '|'.join(user_diets), na=False, regex=False
        )]
    
    if filtered.empty:
        return pd.DataFrame()
    
    # Content-based ranking
    if user_diets:
        user_profile = ' '.join(user_diets)
        user_vector = tfidf.transform([user_profile])
        cos_sim = cosine_similarity(user_vector, tfidf_matrix[filtered.index])
        filtered['similarity'] = cos_sim[0]
        filtered = filtered.sort_values('similarity', ascending=False)
    else:
        # Default sorting for users with no preferences
        filtered = filtered.sort_values(by=['Protein (g)', 'Fiber (g)'], ascending=False)
    
    return filtered.head(n)

# Test for User 3 (no restrictions)
print("Recommendations for User 3:")
display(recommend_meals(23)[['FoodName', 'Calories', 'Common_Allergens', 'Diet_Preference']])

Recommendations for User 3:


,FoodName,Calories,Common_Allergens,Diet_Preference
2,Paksiw na Isda,280,Fish,"Pescatarian, Gluten-Free"
15,Daing na Bangus,190,Fish,"Pescatarian, Gluten-Free"
